In [1]:
import oda_api.token 
import logging
import numpy as np
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve, OdaSpectrum
import matplotlib.pyplot as plt
import astroquery.heasarc
from astropy.wcs import WCS
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.time import Time
from matplotlib.patches import Circle
from astroquery.jplhorizons import Horizons
import pandas as pd
import astropy.units as u
import json
from collections import defaultdict
from astropy.io import ascii
import os

In [2]:
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

Load the ScWs

In [3]:
jupiter_table = ascii.read('../data/jupiter_table.dat')

scw_ids = jupiter_table['scw_id'].data
scw_versions = jupiter_table['scw_ver'].data
scw = [str(id).zfill(12) + ".00" + str(ver) for id, ver in zip(scw_ids, scw_versions)]

start = jupiter_table['start_date'].data
end = jupiter_table['end_date'].data
durations = [(e - s) for e,s in zip(end, start)]

jupiter_ra = jupiter_table['jupiter_ra'].data
jupiter_dec = jupiter_table['jupiter_dec'].data

## QUERYING 

In [4]:
isot_start_times = Time(start, format='mjd').isot
isot_end_times = Time(end, format='mjd').isot
duration_seconds = [duration * 86400 for duration in durations]
year_months = [st[:7] for st in isot_start_times]

15 - 30 keV

In [ ]:
# Already processed ScWs

scw_to_match1 = []
directory1 = "../data/Jupiter/15-30keV/Images"
for filename in os.listdir(directory1):
    if filename.endswith(".fits"):
        scw_to_match1.append(filename[:16])

# scws different from the ones in the table
not_in_table = [s for s in scw_to_match1 if s not in scw]
print("Number of ScWs not in the table:", len(not_in_table))
print("Number of ScWs to query:", len(scw) - (len(scw_to_match1) - len(not_in_table)))

In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:
        
        # Check ScW isn't already in the FITS folder
        if scw_id in scw_to_match1:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "15",
            "E2_keV": "30",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

In [5]:
scw_to_match2 = []
directory2 = "../data/Jupiter/30-60keV/Images"
for filename in os.listdir(directory2):
    if filename.endswith(".fits"):
        scw_to_match2.append(filename[:16])

30 - 60 keV

In [ ]:
from collections import defaultdict

disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for scw_id, start_time, end_time, jra, jdec in filtered_scws:

        if scw_id in scw_to_match2:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "30",
            "E2_keV": "60",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "isgri",
            "osa_version": "OSA11.2",
            "product": "isgri_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    n_complete = sum(1 for _disp in disp_by_date.values() if _disp.is_complete)
    print(f"complete {n_complete} / {len(disp_by_date)}")

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Trying SCW 026700060010.001
Is complete  True
Trying SCW 026700070010.001
Is complete  False
Trying SCW 026700080010.001
Is complete  True
Trying SCW 026700090010.001
Is complete  True
Trying SCW 026700100010.001
Is complete  False
Trying SCW 026700110010.001
Is complete  True
Trying SCW 026700120010.001
Is complete  True
Trying SCW 026700130010.001
Is complete  True
Trying SCW 026700140010.001
Is complete  True
Trying SCW 026700150010.001
Is complete  True
Trying SCW 026700160010.001
Is complete  True
Trying SCW 026700170010.001
Is complete  False
Trying SCW 026700180010.001
Is complete  True
Trying SCW 026700190010.001
Is complete  True
Trying SCW 026700200010.001
Is complete  True
Trying SCW 026700210010.001
Is complete  True
Trying SCW 026700230010.001
Is complete  True
Trying SCW 026700240010.001
Is complete  True
Trying SCW 026700250010.001
Is complete  True
Trying SCW 026700260010.001
Is complete  True
Trying SCW 026700270010.001
Is complete  True
Trying SCW 026700280010.001
Is 

3 - 15 keV

In [ ]:
scw_to_match3 = []
directory3 = "../data/Jupiter/3-15keV/Images"
for filename in os.listdir(directory3):
    if filename.endswith(".fits"):
        scw_to_match3.append(filename[:16])

In [ ]:
jemx_validity = []
for s in scw:
    for row in jupiter_table:
        if str(row['scw_id']).zfill(12) == s[:12]:
            jemx_validity.append(row['jemx_valid'])

In [ ]:
disp_by_date = {}
data_by_date = {}
successful_scws = []

while True:
    image_results = []

    filtered_scws = [
        (scw[i], isot_start_times[i], isot_end_times[i], jupiter_ra[i], jupiter_dec[i])
        for i in range(len(scw))
    ]

    for i, (scw_id, start_time, end_time, jra, jdec) in enumerate(filtered_scws):

        if scw_id in scw_to_match3:
            continue

        if jemx_validity[i] == 0:
            continue

        print(f"Trying SCW {scw_id}")

        par_dict = {
            "RA": jra,
            "DEC": jdec,
            "E1_keV": "3",
            "E2_keV": "15",
            "T_format": "isot",
            "T1": start_time,
            "T2": end_time,
            "detection_threshold": "5",
            "instrument": "jemx",
            "jemx_num": '2',
            "osa_version": "OSA11.2",
            "product": "jemx_image",
            "product_type": "Real",
            "scw_list": [scw_id],
        }

        if scw_id not in disp_by_date:
            disp_by_date[scw_id] = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", wait=False)

        _disp = disp_by_date[scw_id]
        data = data_by_date.get(scw_id, None)

        if data is None and not _disp.is_failed:
            try:
                if not _disp.is_submitted:
                    data = _disp.get_product(**par_dict, silent=True)
                else:
                    _disp.poll()

                print("Is complete ", _disp.is_complete)
                if not _disp.is_complete:
                    continue
                else:
                    data = _disp.get_product(**par_dict, silent=True)
                    data_by_date[scw_id] = data

            except Exception as e:
                print(f"Query failed for SCW {scw_id}: {e}")
                continue

        successful_scws.append(scw_id)
        image_results.append(data)

    if n_complete == len(disp_by_date):
        print("done!")
        break
    else:
        print("not done")

Save FITS

In [ ]:
new_results = []
new_scws = []
for scw_id, data in data_by_date.items():
    if data is not None:
        new_results.append(data)
        new_scws.append(scw_id)

In [ ]:
# Save FITS files
import os

output_dir = "../data/Jupiter/3-15keV/Images/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Check if the directory is empty
#if os.path.exists(output_dir) and os.listdir(output_dir):
   # print(f"Directory '{output_dir}' is not empty. Aborting to prevent overwriting.")
#else:
for i, data in enumerate(new_results):
    #if new_scws[i] in scw_to_match:
    im = OdaImage(data)
    im.write_fits(os.path.join(output_dir, f"{new_scws[i]}"))
